In [1]:
import qiskit as qc
import numpy as np

/home/jupyter-arturolopez/.local/lib/python3.9/site-packages/qiskit/providers/aer/pulse/qutip_extra_lite/fastsparse.py:57: DeprecationWarning: Please use `upcast` from the `scipy.sparse` namespace, the `scipy.sparse.sputils` namespace is deprecated.
  from scipy.sparse.sputils import (upcast, isdense, isscalarlike, get_index_dtype)
/home/jupyter-arturolopez/.local/lib/python3.9/site-packages/qiskit/providers/aer/pulse/qutip_extra_lite/fastsparse.py:57: DeprecationWarning: Please use `isdense` from the `scipy.sparse` namespace, the `scipy.sparse.sputils` namespace is deprecated.
  from scipy.sparse.sputils import (upcast, isdense, isscalarlike, get_index_dtype)
/home/jupyter-arturolopez/.local/lib/python3.9/site-packages/qiskit/providers/aer/pulse/qutip_extra_lite/fastsparse.py:57: DeprecationWarning: Please use `isscalarlike` from the `scipy.sparse` namespace, the `scipy.sparse.sputils` namespace is deprecated.
  from scipy.sparse.sputils import (upcast, isdense, isscalarlike, get_inde

In [2]:
a = qc.QuantumCircuit(3,1)

a.h(0)
a.i(1)
a.cx(0,1)
a.i(2)


a.barrier()

a.y(0)
a.i(1)
a.h(2)

a.cx(1,2)
a.barrier()

a.i(0)
a.x(1)
a.i(2)

a.draw()

┌───┐      ░ ┌───┐      ░ ┌───┐
q_0: ┤ H ├──■───░─┤ Y ├──────░─┤ I ├
     ├───┤┌─┴─┐ ░ ├───┤      ░ ├───┤
q_1: ┤ I ├┤ X ├─░─┤ I ├──■───░─┤ X ├
     ├───┤└───┘ ░ ├───┤┌─┴─┐ ░ ├───┤
q_2: ┤ I ├──────░─┤ H ├┤ X ├─░─┤ I ├
     └───┘      ░ └───┘└───┘ ░ └───┘
c: 1/═══════════════════════════════

In [3]:
def parsing(QuantumCircuit):
    qasm = QuantumCircuit.qasm()
    lis = []
    lin = ""
    for s in qasm:
        if s == ";":
            lis.append(lin)
            lin=""
        elif s == "\n":
            pass
        else:
            lin += s
    
    lis.pop(0)
    lis.pop(0)
    #getting rid of the first two lines of the qasm code
    #could also make a while loop to pop() the lines that don't start with qreg
    
    if "qreg" in lis[0]:
        qreg=lis[0]
        numQubits=int(qreg[-2])
        lis.pop(0)
    else:
        print("There is an error in getting the number of qubits")
    
    if "creg" in lis[0]:
        lis.pop(0)
    
    easyGates = []
    numCycles = 1
    for i in range(len(lis)):
        line = lis[i]
        if "h" in line:
            easyGates.append("H")
        elif "x" in line and "c" not in line:
            easyGates.append("X")
        elif "y" in line:
            easyGates.append("Y")
        elif "z" in line:
            easyGates.append("Z")
        elif "t" in line:
            easyGates.append("T")
        elif "id" in line:
            easyGates.append("I")
        elif "barrier" in line:
            numCycles += 1
        elif "cx" in line:
            continue
        else:
            print(f"unexpected line or gate in qasm interpretation in line {i} ")
    
    controls = []
    targets = []
    
    for i in range(len(lis)):
        line = lis[i]
        if "cx" in line:
            for j in range(len(line)):
                if line[j] == "]":
                    a=line[j-1]
                    if j == 6:
                        controls.append(a)
                    elif j==11:
                        targets.append(a)
                    else:
                        print("something weird happening with cx")
    return numQubits, numCycles, easyGates, controls, targets

In [4]:
(numQubits, numCycles, easyGates, controls, targets)=parsing(a)


In [5]:
#can use structs to store info about circuits

In [6]:
targets

['1', '2']

In [7]:
def twirling_gates(numQubits, numCycles):
    twirling_set = ["X","Y","Z","I"]
    random_twirling = []
    for i in range(numQubits*(numCycles-1)):
        random_twirling.append(np.random.choice(twirling_set))
    for i in range(numQubits):
        random_twirling.append("I")
    return random_twirling

In [8]:
random_twirling = twirling_gates(numQubits, numCycles)
random_twirling

['I', 'Y', 'Y', 'I', 'I', 'Z', 'I', 'I', 'I']

In [9]:
def inverting_operators(random_twirling, targets, numQubits):
    target_indeces = []
    for i in range(len(targets)): #getting indeces of inverting_matrices that need to change
        target_indeces.append(str(int(targets[i])+ numQubits*i))
    
    inverting_gates = []
    for i in range(numQubits):
        inverting_gates.append("I")
    for i in range(len(random_twirling)-numQubits):
        if str(i) in target_indeces:
            #do multiplication formula:: Honestly, nevermind: just match with if state
            if random_twirling[i]=="X":
                inverting_gates.append("I")
            elif random_twirling[i]=="Y":
                inverting_gates.append("idk yet")
            elif random_twirling[i]=="Z":
                inverting_gates.append("idk yet")
            elif random_twirling[i]=="H":
                inverting_gates.append("idk yet")
            elif random_twirling[i]=="T":
                inverting_gates.append("idk yet")
            elif random_twirling[i]=="I":
                inverting_gates.append("idk yet")
            else:
                print("unrecognized gate before cx target qubit in prev. cycle")
        else:
            inverting_gates.append(random_twirling[i])
    return target_indeces, inverting_gates
        

In [10]:
(target_indeces, inverting_gates)=inverting_operators(random_twirling, targets, numQubits)
inverting_gates

['I', 'I', 'I', 'I', 'idk yet', 'Y', 'I', 'I', 'idk yet']

In [11]:
def gate2mat(gate):
    if gate == "X":
        gate_matrix = np.array([[0,1],
                                [1,0]])
    elif gate == "Y":
        gate_matrix = np.array([[0,-1j],
                                [1j,0]])
    elif gate == "Z":
        gate_matrix = np.array([[1,0],
                                [0,-1]])
    elif gate == "H":
        gate_matrix = 1/np.sqrt(2) *np.array([[1,1],
                                [1,-1]])
    elif gate == "I":
        gate_matrix = np.array([[1,0],
                                [0,1]])
    elif gate == "T":
        gate_matrix = np.array([[1,0],
                                [0,np.exp(1j*np.pi/4)]])
    else:
        print(f"matrix {gate} not recognized")
        gate_matrix = np.array([[0,0],[0,0]])
    return gate_matrix

In [12]:
gate2mat("H")

array([[ 0.70710678,  0.70710678],
       [ 0.70710678, -0.70710678]])

In [13]:
mat1=gate2mat("H")
mat2=gate2mat("X")

np.dot(mat1,mat2)

array([[ 0.70710678,  0.70710678],
       [-0.70710678,  0.70710678]])

In [14]:
def reduction(easyGates, random_twirling, inverting_gates, numQubits, numCycles):
    compiled_gates = np.zeros((numQubits*numCycles,2,2),dtype = 'complex_')
    
    for i in range(numQubits*numCycles):
        mat1=gate2mat(inverting_gates[i])
        mat2=gate2mat(easyGates[i])
        mat3=gate2mat(random_twirling[i])
        a=np.dot(mat2,mat1)
        compiled_gates[i]=np.dot(mat3,a)
    
    #want to return an array, matrix of 2x2x(however 
    #many easy gates we have in the circuit= numQubits*numCycles)
    return compiled_gates

In [15]:
(compiled_gates)=reduction(easyGates, random_twirling, inverting_gates, numQubits, numCycles)
compiled_gates

matrix idk yet not recognized
matrix idk yet not recognized


array([[[ 0.70710678+0.j        ,  0.70710678+0.j        ],
        [ 0.70710678+0.j        , -0.70710678+0.j        ]],

       [[ 0.        +0.j        ,  0.        -1.j        ],
        [ 0.        +1.j        ,  0.        +0.j        ]],

       [[ 0.        +0.j        ,  0.        -1.j        ],
        [ 0.        +1.j        ,  0.        +0.j        ]],

       [[ 0.        +0.j        ,  0.        -1.j        ],
        [ 0.        +1.j        ,  0.        +0.j        ]],

       [[ 0.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        +0.j        ]],

       [[ 0.        +0.70710678j,  0.        -0.70710678j],
        [ 0.        +0.70710678j,  0.        +0.70710678j]],

       [[ 1.        +0.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  1.        +0.j        ]],

       [[ 0.        +0.j        ,  1.        +0.j        ],
        [ 1.        +0.j        ,  0.        +0.j        ]],

       [[ 0.        +0.j

In [16]:
#turn matrix into gate: will need to know how to generalize the forms of the
# possible matrices
def mat2gate(a): #given a 2x2 np.array()
    gate=[]
    gateList = ["X", "Y", "Z", "H", "I", "T"]
    
    X = np.array([[0,1],[1,0]])
    Y = np.array([[0,-1j],[1j,0]])
    Z = np.array([[1,0],[0,-1]])
    H = 1/np.sqrt(2) *np.array([[1,1],[1,-1]])
    I = np.array([[1,0],[0,1]])
    T = np.array([[1,0],[0,np.exp(1j*np.pi/4)]])
    
    matList = [X, Y, Z, H, I, T] #only works for these specific definitions
    for i,item in enumerate(matList):
        if (a == item).all():
            gate = gateList[i]
    if gate == []:
        print(f"the following gate needs to be defined:{a}")
    return gate

In [17]:
#yo=np.array([[0,1],[1,1]])
yo=gate2mat("X")
mat2gate(yo)

'X'

In [18]:
def instructions(compiled_gates):
    randEZgates = []
    for i in range(len(compiled_gates)):
        randEZgates.append(mat2gate(compiled_gates[i]))
        
    return randEZgates

In [19]:
randEZgates = instructions(compiled_gates)
print(randEZgates)
print(random_twirling)
print(inverting_gates)
a.draw()

the following gate needs to be defined:[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
the following gate needs to be defined:[[0.+0.70710678j 0.-0.70710678j]
 [0.+0.70710678j 0.+0.70710678j]]
the following gate needs to be defined:[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
['H', 'Y', 'Y', 'Y', [], [], 'I', 'X', []]
['I', 'Y', 'Y', 'I', 'I', 'Z', 'I', 'I', 'I']
['I', 'I', 'I', 'I', 'idk yet', 'Y', 'I', 'I', 'idk yet']


┌───┐      ░ ┌───┐      ░ ┌───┐
q_0: ┤ H ├──■───░─┤ Y ├──────░─┤ I ├
     ├───┤┌─┴─┐ ░ ├───┤      ░ ├───┤
q_1: ┤ I ├┤ X ├─░─┤ I ├──■───░─┤ X ├
     ├───┤└───┘ ░ ├───┤┌─┴─┐ ░ ├───┤
q_2: ┤ I ├──────░─┤ H ├┤ X ├─░─┤ I ├
     └───┘      ░ └───┘└───┘ ░ └───┘
c: 1/═══════════════════════════════

In [21]:
mat1=gate2mat("Y")
mat2=gate2mat("H")
earlier = np.dot(mat2,mat1)
mat3=gate2mat("Z")
earlier
np.dot(mat3,earlier) #-> second gate we gotta add corresponds to |-> state (other ones are from idk yets, meaning we need calculation) 

array([[0.+0.70710678j, 0.-0.70710678j],
       [0.+0.70710678j, 0.+0.70710678j]])

In [159]:
#using the newly made easy gates (from the randomized compiling) and the original 
# hard gates, return a quantum circuit that has the same # of cycles and gates as 
# the bare circuit (and also include barriers)

In [ ]:
def final_product(randEZgates, controls, targets, numQubits):
    qc=QuantumCircuit(numQubits)
    for j in range(len(targets)):
        for i in range(numQubits):
            var = randEZgates[i+j*numQubits]
            if var = "X":
                qc.h(i)
            #all the other gates we could possibly get, including paramaterized gates somehow
        qc.cx(controls[j],targets[j])
        qc.barrier(0,numQubits-1)
            
    #one final loop for last round of randEZgates (not attached to hard gate cycle)
    return qc